In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, normalization, Activation
from keras.layers.normalization import BatchNormalization

from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

C:\Users\Vivek.Sasikumar\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def data():
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    training_set = train_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/training_set',
                                                     target_size=(64, 64),
                                                     batch_size=32,
                                                     class_mode='binary')
    test_set = test_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/test_set',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')
    return training_set, test_set

In [ ]:
def model(X_train,  X_test):
    
    model = Sequential()
    model.add(Conv2D(32, 3,3, activation='relu', input_shape=(64,64,3)))
    model.add(Conv2D(32, 3,3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Conv2D(32, 3,3, activation='relu'))
    model.add(Conv2D(32, 3,3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout({{uniform(0, 1)}}))

    
    model.add(Flatten())
    model.add(Dense({{choice([256, 512])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})
    
    model.fit_generator(X_train,
                        steps_per_epoch=8000,
                        epochs=50,
                        validation_data=X_test,
                        validation_steps=2000)
    
    score, acc = model.evaluate_generator(X_test, steps=2000)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
if __name__ == '__main__':
    best_run, best_model = optim.minimize(model = a_model,
                                          data = data,
                                          algo = tpe.suggest,
                                          max_evals = 2,
                                          trials = Trials(),
                                          notebook_name = 'CNN - Cats Vs Dogs')
    
    X_train, X_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate_generator(X_test, steps=2000))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

In [ ]:
best_model.predict_generator(test_set,steps=1)